In [3]:
from nixtlats import TimeGPT
import pandas as pd
import numpy as np
import torch 
import config
import pickle
from utils import data_handling, helpers
window_size = 96
pred_length = pred_length = (96)

gp_2 = config.CONFIG_OUTPUT_PATH["TimeGPT"] / "TimeGPT_zero_shot_genome_project.pkl"
bavaria = config.CONFIG_OUTPUT_PATH["TimeGPT"] / "TimeGPT_zero_shot_bavaria.pkl"
electricity = config.CONFIG_OUTPUT_PATH["TimeGPT"] / "TimeGPT_zero_shot_electricity.pkl"

results_output_path = config.CONFIG_OUTPUT_PATH["TimeGPT"] / "timegpt_results.csv"

timegpt = TimeGPT(
    # defaults to os.environ.get("TIMEGPT_TOKEN")
    token = 'JxQF7I2vM8Hpbue718L2gGCfG1gflGqWJCkWb6gRjVL2r9p0TPhDmoOIMAAOgGdQG1tFUDFKgkq52cbyAM1H8Li1jTg9KtV6zWKGuejojqMV6i5wzTgppL8KnsvfcfpYidXbBRgqSLlVilfjjvWwTIMFixvHbIu1cEBdP8CUcjeRlsWlg2ZkYlLr3iiG2RojTAG4BKHRnSX2S7uIpaKMjBZcwRlVpf8KoUFXfAaQj2a5UW98o5IMPe196Iz2uqwD'
)

timegpt.validate_token()

INFO:nixtlats.timegpt:Happy Forecasting! :), If you have questions or need support, please email ops@nixtla.io


True

Nixtla's SDK for TimeGPT takes pandas DF as inputs. Here we are using an Electricity Price Forecasting (EPF) dataset.

The input data will require the following columns:

series identifier (column name specified with #id_col argument)
datestamps (column name specified with #time_col argument)
target historical values (column name specified with #target_col argument)

FINE TUNING IS ONLY POSSIBLE STARTING AT 1.5X FORECASTING HORIZON


after forecasting 6 horizons at 96 time steps for 350 IDs, 1000 dollars of forecasting credit are used and no more forecsats can be made.  

In [12]:
def input_tensor_to_gpt_df(input):
    horizon = input.size(0)
    num_ids = input.size(1)


    timestamps = pd.date_range(start='2023-04-01', periods=horizon, freq='H')
    time_steps = np.repeat(timestamps, num_ids)

    ids = np.tile(np.arange(0, num_ids), horizon) 


    input = input.numpy().flatten()


    df = pd.DataFrame({
    'Time': time_steps,
    'ID': ids,
    'Target': input
    })
    
    return df


def reshape_gpt_output_to_eval_tensor(timegpt_fcst_df):
    temp = timegpt_fcst_df

    num_ids = temp["ID"].nunique()
    horizon = temp["Time"].nunique()

    preds = temp["TimeGPT"]
    preds_tensor = torch.tensor(preds.to_list())
    preds_tensor_2d = preds_tensor.view(horizon, num_ids)
    

    return preds_tensor_2d


def forecast_and_save_predictions(data_dict, filename):
    try:
        with open(filename, 'rb') as file:
            # Load data from the file
            prediction_list = pickle.load(file)
    except FileNotFoundError:
        prediction_list = []

    num_96_horizons = int(data_dict["test"].shape[0] / 96)

    print(f"Total number of forecasting horizons: {num_96_horizons}.")


    for i in range(len(prediction_list), num_96_horizons-1):
        time_step = i * 96
        sum_mse = 0
        sum_mae = 0
        print(i)
        input = data_dict["test"][time_step:time_step+96,:]
        target = data_dict["test"][time_step+96: time_step+96+96, :]

        input_df = input_tensor_to_gpt_df(input)
        timegpt_fcst_df = timegpt.forecast(df=input_df, h=96, freq="H", model="timegpt-1-long-horizon", finetune_steps=0, id_col='ID', time_col='Time', target_col='Target')

        preds = reshape_gpt_output_to_eval_tensor(timegpt_fcst_df)

        sum_mse += helpers.mean_squared_error(preds, target)
        sum_mae += helpers.mean_absolute_error(preds, target)


        prediction_list.append([(sum_mse/num_96_horizons).item(), (sum_mae/num_96_horizons).item()])
        print(sum_mse / num_96_horizons)

            # save as pickle
        with open(filename, 'wb') as file:
            pickle.dump(prediction_list, file)

In [37]:
# load electricity dataset split into train/val/test and predict using TimeGPT
data_dict = data_handling.load_electricity()

forecast_and_save_predictions(data_dict, electricity)

[[0.0771762877702713, 0.04084039852023125],
 [0.0771762877702713, 0.04084039852023125],
 [0.07700958847999573, 0.04097707197070122],
 [0.06589864939451218, 0.037898704409599304],
 [0.06078745052218437, 0.036166634410619736],
 [0.06247527897357941, 0.03682928532361984],
 [0.06591875106096268, 0.03770613670349121],
 [0.06986712664365768, 0.03884812071919441],
 [0.06107547506690025, 0.036250289529561996],
 [0.056967686861753464, 0.03498756140470505],
 [0.05675537511706352, 0.034893278032541275],
 [0.0636158436536789, 0.03697675094008446],
 [0.06835135072469711, 0.038090337067842484],
 [0.0595400370657444, 0.03590917959809303],
 [0.060773540288209915, 0.03634771332144737],
 [0.057288601994514465, 0.03478074446320534],
 [0.05759906768798828, 0.034748855978250504],
 [0.05841624736785889, 0.03464595600962639],
 [0.05906099081039429, 0.034901317209005356],
 [0.055539537221193314, 0.034145183861255646],
 [0.056636273860931396, 0.034419432282447815],
 [0.06004079803824425, 0.035513509064912796],

In [41]:
# load bavaria dataset and predict using TimeGPT API
data_tensor = data_handling.load_bavaria_electricity()
bavaria_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

forecast_and_save_predictions(bavaria_dict, bavaria)

INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...


Total number of forecasting horizons: 29.
0


INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...


tensor(0.0138)
1


INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...


...

tensor(0.0290)
26


INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...


tensor(0.0292)
27


INFO:nixtlats.timegpt:Calling Forecast Endpoint...


tensor(0.0294)


In [55]:
# load Genome Project 2 dataset and predict using TimeGPT API
data_tensor = data_handling.load_genome_project_data()
gp_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

forecast_and_save_predictions(gp_dict, gp_2)

INFO:nixtlats.timegpt:Validating inputs...


Total number of forecasting horizons: 30.
0


INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...


tensor(0.0657)
1


INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...


...

tensor(0.0719)
27


INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...


tensor(0.0732)
28


INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Calling Forecast Endpoint...


tensor(0.0728)


# Evaluate

In [4]:
# Open the file in binary read mode
with open(gp_2, 'rb') as file:
    # Load data from the file
    gp_2_preds = pickle.load(file)

# Open the file in binary read mode
with open(electricity, 'rb') as file:
    # Load data from the file
    electricity_preds = pickle.load(file)

# Open the file in binary read mode
with open(bavaria, 'rb') as file:
    # Load data from the file
    bavaria_preds = pickle.load(file)

In [5]:
electricity_mse = [sublist[0] for sublist in electricity_preds]
electricity_mae = [sublist[1] for sublist in electricity_preds]

bavaria_mse = [sublist[0] for sublist in bavaria_preds]
bavaria_mae = [sublist[1] for sublist in bavaria_preds]

gp_2_mse = [sublist[0] for sublist in gp_2_preds]
gp_2_mae = [sublist[1] for sublist in gp_2_preds]

electricity_metrics = sum(electricity_mse) / len(electricity_mse), sum(electricity_mae) / len(electricity_mse)

bavaria_metrics = sum(bavaria_mse) / len(bavaria_mse), sum(bavaria_mae) / len(bavaria_mse)

gp_2_metrics = sum(gp_2_mse) / len(gp_2_mse), sum(gp_2_mae) / len(gp_2_mse)


print(f"Electricity MSE/MAE: {electricity_metrics}\n\
Bavaria MSE/MAE:     {bavaria_metrics}\n\
GP_2 MSE/MAE:        {gp_2_metrics}.")



Electricity MSE/MAE: (0.06677505913479574, 0.037442001032418216)
Bavaria MSE/MAE:     (0.02291652250901929, 0.02199864054897002)
GP_2 MSE/MAE:        (0.08251036896274008, 0.037434005788688).


In [6]:
import pandas as pd
results = pd.DataFrame([electricity_metrics, bavaria_metrics, gp_2_metrics])
results.columns = ["MSE", "MAE"]
results.index = ["ELD", "Bavaria", "GP2"]
results.to_csv(results_output_path)
results

,MSE,MAE
ELD,0.066775,0.037442
Bavaria,0.022917,0.021999
GP2,0.082510,0.037434
